# Vérifier si des publications sont dans la collection HAL d'un laboratoire
Charger les "librairies" (boîtes à outils de code) qui seront nécessaires pour l'opération.
Si nécessaire, les installer d'abord.

In [12]:
import pandas as pd
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog as fd
from functions import check_df,normalise
from halcollimporter import HalCollImporter
tqdm.pandas()
tk.Tk().withdraw()

''

## Informations données par l'utilisateur

Ecrire dans la fenêtre ouverte par la cellule suivante le code de la collection à vérifier

In [13]:
collection_a_chercher=tk.simpledialog.askstring(prompt="Entrez le code de la collection HAL de votre structure",title='Code de la collection').upper()
fichier=fd.askopenfilename(title='Choisissez le fichier qui contient les publications à vérifier')

## Chargement des données à comparer

Choisir dans la fenêtre ouverte par la cellule suivante le fichier contenant les publications à comparer avec HAL. Ce doit être un fichier Excel.

In [14]:
publis_a_checker=pd.read_excel(fichier)
publis_a_checker.rename({"DOI":'doi',"display_name":"Title","Article Title":"Title","Publication Year":"Year"},axis='columns',inplace=True)
publis_a_checker['Statut']=''
if "Year" in publis_a_checker.columns:
    date_debut=int(min(publis_a_checker['Year'].tolist())-1)
    date_fin=int(max(publis_a_checker['Year'].tolist())+1)
else:
    date_debut=None
    date_fin=None

## Import de la collection HAL

Charger depuis HAL les documents de la collection

In [8]:
date_fin

2024

In [15]:
coll=HalCollImporter(collection_a_chercher,date_debut,date_fin)
valid=tk.messagebox.askokcancel(title="Importer les publications ?", message=f"{coll.nbdocs} publications trouvées pour {coll.coll_code} sur la période. Les importer ?")
if valid==True:
    coll_df=coll.import_data()

publications trouvées : 1458
terminé : 1458 publications chargées	

## Traitement principal

Comparer les publications du fichier du labo avec celles issues de HAL. C'est l'opération la plus complexe car chaque ligne fait l'objet de deux comparaisons potentielles : DOI et titre.

In [16]:
coll_df['nti']=coll_df['Titres'].apply(lambda x : normalise(x).strip())
check_df(publis_a_checker,coll_df)

100%|██████████| 279/279 [00:34<00:00,  7.98it/s]


## Export du résultat

Réexporter le fichier du labo. Quatre colonnes ont été ajoutées, "Statut", "Titre si trouvé", "Identifiant HAL si trouvé", et "Type de dépôt si trouvé".
- La colonne "Statut" permet de savoir quelles publications sont déjà dans la collection du labo, lesquelles sont dans HAL mais pas dans la collection (probablement mal affiliées), et lesquelles ne sont pas dans HAL.
- La colonne "Titre si trouvé" permet de comparer le titre trouvé dans HAL avec le titre d'origine du document, pour s'assurer qu'il s'agit bien des mêmes. Cette étape est faite par le code mais une vérification manuelle est souvent utile, en particulier lorsque le statut est "Titre inexact trouvé".
- La colonne "Identifiant HAL si trouvé" permet de vérifier directement dans HAL si certaines informations sont manquantes.
- La colonne "Type de dépôt si trouvé" permet, pour les publications présentes dans HAL, de vérifier s'il s'agit seulement de notices ou si le texte intégral est présent.

In [17]:
publis_a_checker.to_excel(fichier.replace(".xlsx","_traite.xlsx"),index=False)

In [11]:
publis_a_checker[278:282]

,Title,doi,Authors,Author full names,Author(s) ID,Year,Source title,Volume,Issue,Art. No.,...,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID,Statut,titre_si_trouvé,url_hal_si_trouvé
278,Spray Cooling of a Rod Bundle with a Counter-C...,10.13182/NURETH20-40184,Brillant G.,"Brillant, G. (6507826976)",6507826976,2023.0,Proceedings of the 20th International Topical ...,NaN,NaN,NaN,...,English,Proc. Int. Top. Meet. Nucl. React. Therm. Hydr...,Conference paper,Final,NaN,Scopus,2-s2.0-85202339824,,NaN,NaN
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN


In [18]:
coll_df.columns

Index(['Hal_ids', 'DOIs', 'Titres', 'Types de dépôts', 'nti'], dtype='object')

In [23]:
publis_a_checker['Title'][8]

'Use of the new dynamic cone penetrometer for the study of soil liquefaction along the Kupa River\\, Petrinja area (Croatia)'

In [22]:
coll_df['Titres'][67]

'Use of the new dynamic cone penetrometer for the study of soil liquefaction along the Kupa River, Petrinja area (Croatia)'